In [ ]:
import numpy as np
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm

# 학습 데이터 만들기

In [ ]:
minwon_df = pd.read_csv('..\\data\\0. 새올_통합.csv')
minwon_df

,req_id,req_title,req_p,req_date,req_content,resp_dept,resp_date,resp_writer,resp_content,page_num,gu_name,국,하는일
0,445530.0,양재천 너구리(?) 출현,홍영규,2023-07-31 08:42:53,어제 양재천 오후 9:30분 쯤 계양 중학교 후면 산책을 하다가 숲속에 너구리 여러...,공원녹지과,2023-08-01 18:01:12,이지영,"안녕하십니까? 우리 구정에 많은 관심과 애정을 가져주심에 감사드리며, 양재천에 서식...",1,강남구,도시환경국,근린 주제공원 조성 변경 에 관한 도시계획 업무 총괄 도시관리 변경 계획 공원조성 ...
1,444484.0,통신판매업 신고 - 여러 사이트의 경우,김○○,2023-07-27 10:56:28,안녕하세요\n통신판매업 신고가 이미 되어있는 상태입니다.(a쇼핑몰 주소)\n같은 통...,지역경제과,2023-08-01 18:07:52,김나연,"1. 우리구정에 깊은 관심과 애정을 보여 주신데 대하여 감사드리며, 접수번호(202...",1,강남구,기획경제국,경제동향 분석 세곡천 힐링텃밭 운영 및 관리 스마트팜 조성 및 운영 과 주요사업 및...
2,443723.0,도로 노면 홈으로 인한 물튀김 피해,O○○,2023-07-24 21:19:42,지난 화요일(7/18) 오전 8:50~8:55분경 압구정로 30길 I.S.A빌딩과 ...,도로관리과,2023-08-01 16:46:45,최종현,1. 안녕하십니까. 먼저 우리 구 도로에서 물튀김 피해를 입으신 것에 대해 안타깝게...,1,강남구,안전교통국,구의회 업무 총무 인사 보안 직원복무 교육 복리후생 급여 및 제수당 청렴 물품관리 ...
3,443578.0,실내 공기질 측정기 대여 서비스,김○○,2023-07-24 14:38:41,연일 노고에 감사드립니다\n\n올해 개포동에 신축아파트들이 많이 지어집니다\n실내공...,환경과,2023-07-28 08:52:35,노서우,안녕하십니까? 평소 구정 발전에 적극 협조하여 주신 귀하께 깊이 감사드립니다. 귀하...,1,강남구,도시환경국,그린 스마트 쉼터 신설 및 관리 압구정로 도산대로 학동로 기후변화 취약계층 지원사업...
4,442559.0,이동약자의 이동공간 확보를 도와주세요,정○○,2023-07-21 10:37:24,붙임 사진을 제 아버님이 거주하시는 개포우성2차 아파트 출입구 사진입니다.\n보시면...,주택과,2023-07-31 17:38:48,김혜숙,1.안녕하십니까? 귀하께서 신청하신 민원에 대해 다음과 같이 안내드립니다.\n2.귀...,1,강남구,도시환경국,주요업무계획 수립 국예산 및 결산 국 장 회의자료 국 인사 근평 국 소관 구의회 업...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
88034,1136.0,불법주차가 너무 심해서 차가 너무 막혀요,박종수,2008-01-13 00:02:53,불법주차를 가장 잘 단속해야 하는 지역이 중랑구청 앞이란 당연한 생각이 듭니다.\n...,교통행정과,2008-01-18 13:17:45,김충만,○ 항상 구정발전에 관심을 가져주신 고객님의 가정에 행운이 가득하시길 기원합니다.\...,145,중랑구,안전건설교통국,교통유발부담금 화물인허가 택시 인허가 어린이보호구역 개선사업 교통체계개선사업 교통체...
88035,1135.0,어이없어서 이렇게 글을 올립니다.,문수철,2008-01-11 17:10:41,안녕하세요\n면목5동 에서 슈퍼를 운영하고 있는 사람입니다.\n다름이 아니라 슈퍼를...,지역경제과,2008-01-18 14:18:25,이종희,○ 구 행정에 많은 관심과 협조를 해주시는 문수철님께 감사를 드립니다.\n○ 귀하의...,145,중랑구,기획재정국,골목형상점가 시설현대화 사업 공모사업 및 공사 등 대규모 및 준대규모점포 골목상권 ...
88036,1124.0,중랑구 빈병회수 에대한 법 22조 2항을 위반한 업체들을 처분해 주십시요.,강상길,2007-12-31 13:09:20,공병회수 에 대해 환경부에 밑에 민원상담을 했습니다.\n\n저히집은 애주가 집안이라...,청소행정과,2008-01-03 09:15:38,김용식,- 고객님 안녕하십니까?\n\n- 먼저 고객님의 빈병 보증금 회수 관계로 불편을 드...,145,중랑구,생활복지국,종량제 봉투 제작 및 공급 세외수입 중간집하장 이전 폐기물처리시설 건립기금관리 재활...
88037,1112.0,길고양이 TNR관련,백송이,2007-11-29 10:23:33,안녕하세요.\n내년부터 서울시에서 길고양이 TNR을 실시한다는 얘기를 들었습니다.\...,지역경제과,2007-12-04 15:08:27,김종수,안녕하세요?\n고양이 문제에 대하여 많은 애정과 관심을 가지고 좋은 의견을 제시해 ...,145,중랑구,기획재정국,골목형상점가 시설현대화 사업 공모사업 및 공사 등 대규모 및 준대규모점포 골목상권 ...


In [ ]:
minwon_df['req_title_content'] = minwon_df['req_title'] + ' ' + minwon_df['req_content']
minwon_df

,req_id,req_title,req_p,req_date,req_content,resp_dept,resp_date,resp_writer,resp_content,page_num,gu_name,국,하는일,req_title_content
0,445530.0,양재천 너구리(?) 출현,홍영규,2023-07-31 08:42:53,어제 양재천 오후 9:30분 쯤 계양 중학교 후면 산책을 하다가 숲속에 너구리 여러...,공원녹지과,2023-08-01 18:01:12,이지영,"안녕하십니까? 우리 구정에 많은 관심과 애정을 가져주심에 감사드리며, 양재천에 서식...",1,강남구,도시환경국,근린 주제공원 조성 변경 에 관한 도시계획 업무 총괄 도시관리 변경 계획 공원조성 ...,양재천 너구리(?) 출현 어제 양재천 오후 9:30분 쯤 계양 중학교 후면 산책을 ...
1,444484.0,통신판매업 신고 - 여러 사이트의 경우,김○○,2023-07-27 10:56:28,안녕하세요\n통신판매업 신고가 이미 되어있는 상태입니다.(a쇼핑몰 주소)\n같은 통...,지역경제과,2023-08-01 18:07:52,김나연,"1. 우리구정에 깊은 관심과 애정을 보여 주신데 대하여 감사드리며, 접수번호(202...",1,강남구,기획경제국,경제동향 분석 세곡천 힐링텃밭 운영 및 관리 스마트팜 조성 및 운영 과 주요사업 및...,통신판매업 신고 - 여러 사이트의 경우 안녕하세요\n통신판매업 신고가 이미 되어있는...
2,443723.0,도로 노면 홈으로 인한 물튀김 피해,O○○,2023-07-24 21:19:42,지난 화요일(7/18) 오전 8:50~8:55분경 압구정로 30길 I.S.A빌딩과 ...,도로관리과,2023-08-01 16:46:45,최종현,1. 안녕하십니까. 먼저 우리 구 도로에서 물튀김 피해를 입으신 것에 대해 안타깝게...,1,강남구,안전교통국,구의회 업무 총무 인사 보안 직원복무 교육 복리후생 급여 및 제수당 청렴 물품관리 ...,도로 노면 홈으로 인한 물튀김 피해 지난 화요일(7/18) 오전 8:50~8:55분...
3,443578.0,실내 공기질 측정기 대여 서비스,김○○,2023-07-24 14:38:41,연일 노고에 감사드립니다\n\n올해 개포동에 신축아파트들이 많이 지어집니다\n실내공...,환경과,2023-07-28 08:52:35,노서우,안녕하십니까? 평소 구정 발전에 적극 협조하여 주신 귀하께 깊이 감사드립니다. 귀하...,1,강남구,도시환경국,그린 스마트 쉼터 신설 및 관리 압구정로 도산대로 학동로 기후변화 취약계층 지원사업...,실내 공기질 측정기 대여 서비스 연일 노고에 감사드립니다\n\n올해 개포동에 신축아...
4,442559.0,이동약자의 이동공간 확보를 도와주세요,정○○,2023-07-21 10:37:24,붙임 사진을 제 아버님이 거주하시는 개포우성2차 아파트 출입구 사진입니다.\n보시면...,주택과,2023-07-31 17:38:48,김혜숙,1.안녕하십니까? 귀하께서 신청하신 민원에 대해 다음과 같이 안내드립니다.\n2.귀...,1,강남구,도시환경국,주요업무계획 수립 국예산 및 결산 국 장 회의자료 국 인사 근평 국 소관 구의회 업...,이동약자의 이동공간 확보를 도와주세요 붙임 사진을 제 아버님이 거주하시는 개포우성2...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88034,1136.0,불법주차가 너무 심해서 차가 너무 막혀요,박종수,2008-01-13 00:02:53,불법주차를 가장 잘 단속해야 하는 지역이 중랑구청 앞이란 당연한 생각이 듭니다.\n...,교통행정과,2008-01-18 13:17:45,김충만,○ 항상 구정발전에 관심을 가져주신 고객님의 가정에 행운이 가득하시길 기원합니다.\...,145,중랑구,안전건설교통국,교통유발부담금 화물인허가 택시 인허가 어린이보호구역 개선사업 교통체계개선사업 교통체...,불법주차가 너무 심해서 차가 너무 막혀요 불법주차를 가장 잘 단속해야 하는 지역이 ...
88035,1135.0,어이없어서 이렇게 글을 올립니다.,문수철,2008-01-11 17:10:41,안녕하세요\n면목5동 에서 슈퍼를 운영하고 있는 사람입니다.\n다름이 아니라 슈퍼를...,지역경제과,2008-01-18 14:18:25,이종희,○ 구 행정에 많은 관심과 협조를 해주시는 문수철님께 감사를 드립니다.\n○ 귀하의...,145,중랑구,기획재정국,골목형상점가 시설현대화 사업 공모사업 및 공사 등 대규모 및 준대규모점포 골목상권 ...,어이없어서 이렇게 글을 올립니다. 안녕하세요\n면목5동 에서 슈퍼를 운영하고 있는 ...
88036,1124.0,중랑구 빈병회수 에대한 법 22조 2항을 위반한 업체들을 처분해 주십시요.,강상길,2007-12-31 13:09:20,공병회수 에 대해 환경부에 밑에 민원상담을 했습니다.\n\n저히집은 애주가 집안이라...,청소행정과,2008-01-03 09:15:38,김용식,- 고객님 안녕하십니까?\n\n- 먼저 고객님의 빈병 보증금 회수 관계로 불편을 드...,145,중랑구,생활복지국,종량제 봉투 제작 및 공급 세외수입 중간집하장 이전 폐기물처리시설 건립기금관리 재활...,중랑구 빈병회수 에대한 법 22조 2항을 위반한 업체들을 처분해 주십시요. 공병회수...
88037,1112.0,길고양이 TNR관련,백송이,2007-11-29 10:23:33,안녕하세요.\n내년부터 서울시에서 길고양이 TNR을 실시한다는 얘기를 들었습니다.\...,지역경제과,2007-12-04 15:08:27,김종수,안녕하세요?\n고양이 문제에 대하여 많은 애정과 관심을 가지고 좋은 의견을 제시해 ...,145,중랑구,기획재정국,골목형상점가 시설현대화 사업 공모사업 및 공사 등 대규모 및 준대규모점포 골목상권 ...,길고양이 TNR관련 안녕하세요.\n내년부터 서울시에서 길고양이 TNR을 실시한다는 ...


In [ ]:
# 토픽 모델링을 거친 '과 하는일' 데이터
df = pd.read_csv('..\\data\\0. 구_과_하는일_통합.csv')
df

,구,국,과,하는일,ex_token,ex_text,token,text,topic
0,광진구,행정국,총무과,광진구 의전지침 수립 및 현안 관리 등 신년인사회 구민의 날 정례조례 등 구 주요행...,"['광진구', '의전', '지침', '현안', '신년', '사회', '구민', '정...",광진구 의전 지침 현안 신년 사회 구민 정례 조례 행사 광진구 방위 협의회 일상 경...,"['광진구', '의전', '지침', '수립', '현안', '관리', '신년', '사...",광진구 의전 지침 수립 현안 관리 신년 사회 구민 정례 조례 주요 행사 추진 광진구...,32
1,광진구,행정국,자치행정과,동 행정 종합계획 동 직원 근무평정 조정 고향사랑기부제 동 지역책임제 사업 동 행정...,"['행정', '근무', '평정', '조정', '고향', '사랑', '기부', '책임...",행정 근무 평정 조정 고향 사랑 기부 책임 행정실 커뮤니티 공간 조성 적십자사 선거...,"['행정', '종합', '계획', '직원', '근무', '평정', '조정', '고향...",행정 종합 계획 직원 근무 평정 조정 고향 사랑 기부 지역 책임 사업 행정실 종합 ...,34
2,광진구,행정국,문화예술과,나루아트센터 리모델링 광진문화재단 지원 충무 보안 통합문화이용권 문화도시 사회복무요...,"['나루', '아트센터', '리모델링', '광진', '문화', '재단', '충무',...",나루 아트센터 리모델링 광진 문화 재단 충무 보안 문화 이용권 문화 도시 사회복무요...,"['나루', '아트센터', '리모델링', '광진', '문화', '재단', '지원',...",나루 아트센터 리모델링 광진 문화 재단 지원 충무 보안 통합 문화 이용권 문화 도시...,33
3,광진구,행정국,체육진흥과,체육인 통합 커뮤니티센터 및 구민 체육 증진센터 조성 장애인체육회 설립 자양체육관 ...,"['체육인', '커뮤니티', '구민', '체육', '증진', '조성', '장애인',...",체육인 커뮤니티 구민 체육 증진 조성 장애인 체육회 설립 자양 체육관 스포츠 클럽 ...,"['체육인', '통합', '커뮤니티', '센터', '구민', '체육', '증진', ...",체육인 통합 커뮤니티 센터 구민 체육 증진 센터 조성 장애인 체육회 설립 자양 체육...,12
4,광진구,행정국,교육지원과,구립도서관 국시비보조사업 다문화서비스 서울북스타트 도서관과 함께 책읽기 독서동아리 ...,"['구립', '도서관', '시비', '보조', '다문화', '서비스', '스타트',...",구립 도서관 시비 보조 다문화 서비스 스타트 도서관 독서 동아리 구립 도서관 일자리...,"['구립', '도서관', '시비', '보조', '사업', '다문화', '서비스', ...",구립 도서관 시비 보조 사업 다문화 서비스 서울 스타트 도서관 독서 동아리 운영 지...,25
...,...,...,...,...,...,...,...,...,...
929,노원구,보건소,생활보건과,방역소독사업 관련 업무 소독의무시설 소독업소 포함 관리 직원 복무 및 수당 에이즈 ...,"['방역', '소독', '소독', '의무', '소독', '업소', '포함', '복무...",방역 소독 소독 의무 소독 업소 포함 복무 수당 에이즈 매개 감염병 선별 진료소 선...,"['방역', '소독', '사업', '관련', '업무', '소독', '의무', '시설...",방역 소독 사업 관련 업무 소독 의무 시설 소독 업소 포함 관리 직원 복무 수당 에...,3
930,노원구,보건소,의약과,대직자 강미숙 재난안전관리에 관한 사항 구급차 응급의료기관 관리 장기기증 헌혈사랑 ...,"['대직자', '미숙', '재난', '안전', '구급차', '응급', '의료', '...",대직자 미숙 재난 안전 구급차 응급 의료 장기 기증 헌혈 사랑 안경 치과 공소 의료...,"['대직자', '미숙', '재난', '안전', '관리', '사항', '구급차', '...",대직자 미숙 재난 안전 관리 사항 구급차 응급 의료 기관 관리 장기 기증 헌혈 사랑...,3
931,노원구,보건소,보건지소,보건지소 총괄 업무 월계 공릉지소 예산 회계 직원 수당관리 구강보건사업 전반 한의약...,"['보건', '지소', '공릉', '지소', '수당', '구강', '보건', '전반...",보건 지소 공릉 지소 수당 구강 보건 전반 의약 건강 증진 한방 진료 상담 의약 건...,"['보건', '지소', '총괄', '업무', '공릉', '지소', '예산', '회계...",보건 지소 총괄 업무 공릉 지소 예산 회계 직원 수당 관리 구강 보건 사업 전반 의...,3
932,도봉구,미래환경국,자원순환과,자원순환과 업무 총괄자원순환팀 업무 총괄과서무 예산 회계청소차량 정비 및 정비시설 ...,"['자원', '순환', '자원', '순환', '서무', '청소', '정비', '정비...",자원 순환 자원 순환 서무 청소 정비 정비 봉투 판매소 지정 취소 공중화장실 종량제...,"['자원', '순환', '업무', '총괄', '자원', '순환', '업무', '총괄...",자원 순환 업무 총괄 자원 순환 업무 총괄 서무 예산 회계 청소 차량 정비 정비 시...,11


In [ ]:
topic_df = df[['구', '과', 'topic']]
topic_df.columns = ['gu_name', 'resp_dept', 'topic']
topic_df

,gu_name,resp_dept,topic
0,광진구,총무과,32
1,광진구,자치행정과,34
2,광진구,문화예술과,33
3,광진구,체육진흥과,12
4,광진구,교육지원과,25
...,...,...,...
929,노원구,생활보건과,3
930,노원구,의약과,3
931,노원구,보건지소,3
932,도봉구,자원순환과,11


In [ ]:
merge_df = pd.merge(minwon_df, topic_df, on=['gu_name', 'resp_dept'])
merge_df

,req_id,req_title,req_p,req_date,req_content,resp_dept,resp_date,resp_writer,resp_content,page_num,gu_name,국,하는일,req_title_content,topic
0,445530.0,양재천 너구리(?) 출현,홍영규,2023-07-31 08:42:53,어제 양재천 오후 9:30분 쯤 계양 중학교 후면 산책을 하다가 숲속에 너구리 여러...,공원녹지과,2023-08-01 18:01:12,이지영,"안녕하십니까? 우리 구정에 많은 관심과 애정을 가져주심에 감사드리며, 양재천에 서식...",1,강남구,도시환경국,근린 주제공원 조성 변경 에 관한 도시계획 업무 총괄 도시관리 변경 계획 공원조성 ...,양재천 너구리(?) 출현 어제 양재천 오후 9:30분 쯤 계양 중학교 후면 산책을 ...,16
1,440728.0,갈매기공원 도랑 수리 등,양세성,2023-07-15 11:22:49,1. 8차 현대아파트 옆 공원 물빠짐 도랑 수리\n도수관이 여기저기 불거져 물이 고...,공원녹지과,2023-07-25 13:10:01,정우영,1. 우리 구정발전을 위하여 많은 관심과 애정을 가져주신데 감사드립니다.\n\n2....,1,강남구,도시환경국,근린 주제공원 조성 변경 에 관한 도시계획 업무 총괄 도시관리 변경 계획 공원조성 ...,갈매기공원 도랑 수리 등 1. 8차 현대아파트 옆 공원 물빠짐 도랑 수리\n도수관이...,16
2,437057.0,"가로등 불빛을 가리는 나무를 전지, 전정해 주세요",백승도,2023-07-04 14:37:02,1. 세곡교에서 강남신동아파밀리에1단지로 들어오는 보행로상에 가로등이 설치되어 있으...,공원녹지과,2023-07-07 11:23:24,이지영,"1. 우리 구정에 깊은 관심을 가져주심에 감사드리며, 세곡1교에서 신동아파밀리에 1...",2,강남구,도시환경국,근린 주제공원 조성 변경 에 관한 도시계획 업무 총괄 도시관리 변경 계획 공원조성 ...,"가로등 불빛을 가리는 나무를 전지, 전정해 주세요 1. 세곡교에서 강남신동아파밀리에...",16
3,436489.0,급식소 철거 계고장을 철회해 주세요,정미혜,2023-07-02 22:09:50,서울시 강남구 청담동 나들목 급식소 철거 계고장에 대한 원만한 해결을 부탁드립니다....,공원녹지과,2023-07-11 08:42:18,이지현,"1. 우리 구정 발전을 위하여 많은 관심과 애정을 가져주신데 감사드리며, 청담나들목...",2,강남구,도시환경국,근린 주제공원 조성 변경 에 관한 도시계획 업무 총괄 도시관리 변경 계획 공원조성 ...,급식소 철거 계고장을 철회해 주세요 서울시 강남구 청담동 나들목 급식소 철거 계고장...,16
4,431715.0,고압선 접촉 가로수 가지치기,권오민,2023-06-15 09:25:42,"가로수가 고압선 전선에 접촉해 있습니다.\n\n정전, 단선의 위험이 있으니 빨리 가...",공원녹지과,2023-06-22 09:44:17,조성현,1. 항상 우리 구정 발전에 많은 관심을 가져주심에 감사드립니다.\n2. 귀하께서 ...,3,강남구,도시환경국,근린 주제공원 조성 변경 에 관한 도시계획 업무 총괄 도시관리 변경 계획 공원조성 ...,"고압선 접촉 가로수 가지치기 가로수가 고압선 전선에 접촉해 있습니다.\n\n정전, ...",16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88034,61556.0,코로나19에 대응할 살균소독제를 무상으로 기부하겠습니다. (살균소독제 제조업체),박인태,2020-03-03 13:37:23,1. 본 민원인은 살균 소독제를 제조하고 있습니다.\n\n2. 살균 소독제의 제품명...,위생과,2020-03-06 17:32:52,공용기,"안녕하십니까?\n코로나19 감염 확산을 막기 위해 우리구는 선별진료소, 터미널, 지...",63,중랑구,보건소,식품진흥기금 음식문화개선사업 행정심판 및 행정소송 음식점 원산지 관리 음식점 원산지...,코로나19에 대응할 살균소독제를 무상으로 기부하겠습니다. (살균소독제 제조업체) 1...,10
88035,48831.0,상봉동 봉우재로 33길 음식점에서 길에다 노상피고 못살겠습니다.,정○○,2019-07-03 13:49:33,봉우재로 33길에있는 음식점들 노상(서울시 주차장자리)에 가판피고 음식팔고 술팔고 ...,위생과,2019-07-11 10:25:45,고영배,안녕하십니까?\n평소 우리 구정에 많은 관심과 애정을 가져주셔서 감사합니다.\n중랑...,72,중랑구,보건소,식품진흥기금 음식문화개선사업 행정심판 및 행정소송 음식점 원산지 관리 음식점 원산지...,상봉동 봉우재로 33길 음식점에서 길에다 노상피고 못살겠습니다. 봉우재로 33길에있...,10
88036,48540.0,중랑역 포장마차 위생상태 현황 점검 여부 및 카드결제 거부로 인한 현금결제 강요,김○○,2019-06-26 17:17:43,"본인은 일시적인 내용을 보고, 야기 하는 것이 아닌 중화동에서 초,중,고를 나온만큼...",위생과,2019-07-03 13:19:35,고영배,안녕하십니까?\n평소 우리 구정에 많은 관심과 애정을 가져주셔서 감사합니다.\n현재...,73,중랑구,보건소,식품진흥기금 음식문화개선사업 행정심판 및 행정소송 음식점 원산지 관리 음식점 원산지...,중랑역 포장마차 위생상태 현황 점검 여부 및 카드결제 거부로 인한 현금결제 강요 본...,10
88037,80967.0,중랑구 치안을 위한 중랑구청 차원의 근본적인 정책시행바랍니다.,윤새나,2021-01-04 09:50:00,"오늘난 기사입니다.. 전국 안전지수 하위 지역에 서울 중 가장 낮은 지역, 전국으로...",기획예산과,2021-01-28 13:49:27,박은정,○ 안녕하십니까? 구정에 많은 관심과 애정을 가지고 소중한 의견을 주신 귀하께 감사...,43,중랑구,기획재정국,지속가능발전 기본전략 및 추진계획 국서무 당정협의회 구정질의 핵심지표 개발 및 평가...,중랑구 치안을 위한 중랑구청 차원의 근본적인 정책시행바랍니다. 오늘난 기사입니다.....,34


In [ ]:
# 필요한 컬럼만 가지고 학습 데이터 만들기

train_col = ['req_title_content', 'resp_dept', 'gu_name', 'topic']
train_data = merge_df[train_col]
train_data

,req_title_content,resp_dept,gu_name,topic
0,양재천 너구리(?) 출현 어제 양재천 오후 9:30분 쯤 계양 중학교 후면 산책을 ...,공원녹지과,강남구,16
1,갈매기공원 도랑 수리 등 1. 8차 현대아파트 옆 공원 물빠짐 도랑 수리\n도수관이...,공원녹지과,강남구,16
2,"가로등 불빛을 가리는 나무를 전지, 전정해 주세요 1. 세곡교에서 강남신동아파밀리에...",공원녹지과,강남구,16
3,급식소 철거 계고장을 철회해 주세요 서울시 강남구 청담동 나들목 급식소 철거 계고장...,공원녹지과,강남구,16
4,"고압선 접촉 가로수 가지치기 가로수가 고압선 전선에 접촉해 있습니다.\n\n정전, ...",공원녹지과,강남구,16
...,...,...,...,...
88034,코로나19에 대응할 살균소독제를 무상으로 기부하겠습니다. (살균소독제 제조업체) 1...,위생과,중랑구,10
88035,상봉동 봉우재로 33길 음식점에서 길에다 노상피고 못살겠습니다. 봉우재로 33길에있...,위생과,중랑구,10
88036,중랑역 포장마차 위생상태 현황 점검 여부 및 카드결제 거부로 인한 현금결제 강요 본...,위생과,중랑구,10
88037,중랑구 치안을 위한 중랑구청 차원의 근본적인 정책시행바랍니다. 오늘난 기사입니다.....,기획예산과,중랑구,34


In [ ]:
# 컬럼 이름 변경
train_data.columns = ['req_content', 'resp_dept', 'gu_name', 'topic']
train_data

,req_content,resp_dept,gu_name,topic
0,양재천 너구리(?) 출현 어제 양재천 오후 9:30분 쯤 계양 중학교 후면 산책을 ...,공원녹지과,강남구,16
1,갈매기공원 도랑 수리 등 1. 8차 현대아파트 옆 공원 물빠짐 도랑 수리\n도수관이...,공원녹지과,강남구,16
2,"가로등 불빛을 가리는 나무를 전지, 전정해 주세요 1. 세곡교에서 강남신동아파밀리에...",공원녹지과,강남구,16
3,급식소 철거 계고장을 철회해 주세요 서울시 강남구 청담동 나들목 급식소 철거 계고장...,공원녹지과,강남구,16
4,"고압선 접촉 가로수 가지치기 가로수가 고압선 전선에 접촉해 있습니다.\n\n정전, ...",공원녹지과,강남구,16
...,...,...,...,...
88034,코로나19에 대응할 살균소독제를 무상으로 기부하겠습니다. (살균소독제 제조업체) 1...,위생과,중랑구,10
88035,상봉동 봉우재로 33길 음식점에서 길에다 노상피고 못살겠습니다. 봉우재로 33길에있...,위생과,중랑구,10
88036,중랑역 포장마차 위생상태 현황 점검 여부 및 카드결제 거부로 인한 현금결제 강요 본...,위생과,중랑구,10
88037,중랑구 치안을 위한 중랑구청 차원의 근본적인 정책시행바랍니다. 오늘난 기사입니다.....,기획예산과,중랑구,34


In [ ]:
import os

os.listdir('..\\data')

['0. 구_과_하는일_통합.csv',
 '0. 새올_통합.csv',
 'droped data.csv',
 'raw data',
 '과 하는일 tidy data']

In [ ]:
train_data.to_csv('..\\data\\train_data.csv', index=False, encoding='utf-8-sig')